In [5]:
# 연재 중 웹툰 크롤링

from bs4 import BeautifulSoup
from pprint import pprint
import requests
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import pymysql
import re
from urllib.parse import urlparse, parse_qs

# mysql db connect
# need to fill below host, user, password
#db = pymysql.connect(host='', user='', password='', port=3306, db='readers', charset='utf8', cursorclass=pymysql.cursors.DictCursor)
cursor1 = db.cursor()
cursor2 = db.cursor()
cursor3 = db.cursor()
cursor4 = db.cursor()


In [2]:
# ------ 1. 연재 웹툰 크롤링

sel_sql = "select toon_id from toon_info"
ins_sql = "insert ignore into toon_info(toon_id, toon_name, toon_url, toon_desc, toon_site, wrt_name, is_end, toon_thumb_url)\
           values (%s, %s, %s, %s, %s, %s, %s, %s)"
weekday_sql = "insert ignore into toon_weekday(toon_id, toon_weekday)\
               values (%s, %s)"
genre_sql = "insert ignore into toon_genre(toon_id, genre_name)\
             values (%s, %s)"

#need to fill below webdriver path
#browser = webdriver.Chrome('C:/.../chromedriver.exe')
browser.get('https://www.mrblue.com/webtoon/all')

for day in range(1, 8):
    print('*********************')
    print(day, 'day')
    
    for num in range(1, 100):
        try:
            daylist = browser.find_element_by_xpath("//*[@id=\"contents\"]/div[2]/div/ul/li["+ str(day) +"]")
            # browser.implicitly_wait(3)
            webtoon = daylist.find_element_by_xpath("div/div/ul/li["+ str(num) +"]/div[1]/a")
            #browser.execute_script("arguments[0].scrollIntoView(true);", webtoon) # Scroll to this webtoon element
            
            
            # 썸네일 주소
            thumb = webtoon.find_element_by_xpath('img')
            toon_thumb_url = thumb.get_attribute('src')
                # 성인 작품 여부 판단해서 pass
            if '_adult' in toon_thumb_url:
                num +=1
                continue
            toon_thumb_url = thumb.get_attribute('src')
            
            # url
            toon_url = webtoon.get_attribute('href')
            
            # ID
            toon_id = 'mrblue_' + str(toon_url).replace("https://www.mrblue.com/webtoon/", '')
            
            # 요일 추출
            if day == 1: weekday = 'mon'
            if day == 2: weekday = 'tue'
            if day == 3: weekday = 'wed'
            if day == 4: weekday = 'thu'
            if day == 5: weekday = 'fri'
            if day == 6: weekday = 'sat'
            if day == 7: weekday = 'sun'
            cursor1.execute(weekday_sql, (toon_id, weekday))
            
            # 제목
            toon_name = thumb.get_attribute('alt')
            
            # 장르
            webtoon = daylist.find_element_by_xpath("div/div/ul/li["+ str(num) +"]/div[2]/span[2]")
            genres = webtoon.find_element_by_xpath("span/a")
            genre = genres.get_attribute('title')
            
            # 작가
            wrts = webtoon.find_element_by_xpath("a")
            wrt_name = wrts.get_attribute('title')
                # 글/그림작가 다른 경우
            try:
                wrts = webtoon.find_element_by_xpath("a[2]")
                wrt_name += ', ' + wrts.get_attribute('title')
            except:
                pass
            
            
            # 설명
            ''' # 작품 상세 페이지로 들어가서 description 긁어와야하는데
            # 시간도 많이 걸리는 데다 텍스트가 넘 길고 설명 외에 다른 공지사항이 많아서 걍 뺌
            browser.get(toon_url)
            description = browser.find_element_by_class_name('txt-box')
            desc = description.find_element_by_xpath("p/span")
            toon_desc = desc.text
            
            browser.get('https://www.mrblue.com/webtoon/all')

            '''
            toon_desc = ''
            
            # toon_info
            cursor1.execute(sel_sql)
            result = cursor1.fetchall()
            if (toon_id not in result):
                cursor2.execute(ins_sql, (toon_id, toon_name, toon_url, toon_desc, 'mrblue', wrt_name, 'X', toon_thumb_url))
                cursor3.execute(genre_sql, (toon_id, genre))
            db.commit()
            
        except NoSuchElementException:
            break

*********************
1 day
*********************
2 day
*********************
3 day
*********************
4 day


c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'mrblue_wt_w_divorce_wz' for key 'PRIMARY'")
  self._do_get_result()
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'mrblue_wt_w_divorce_wz-드라마' for key 'PRIMARY'")
  self._do_get_result()


*********************
5 day


c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'mrblue_wt_revenge_wz' for key 'PRIMARY'")
  self._do_get_result()
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'mrblue_wt_revenge_wz-로맨스' for key 'PRIMARY'")
  self._do_get_result()


*********************
6 day
*********************
7 day


In [3]:
# ------ 2. 완결 웹툰 크롤링

sel_sql = "select toon_id from toon_info"
ins_sql = "insert ignore into toon_info(toon_id, toon_name, toon_url, toon_desc, toon_site, wrt_name, is_end, toon_thumb_url)\
           values (%s, %s, %s, %s, %s, %s, %s, %s)"
genre_sql = "insert ignore into toon_genre(toon_id, genre_name)\
             values (%s, %s)"


#need to fill below webdriver path
#browser = webdriver.Chrome('C:/.../chromedriver.exe')
browser.get('https://www.mrblue.com/webtoon/completed')

for num in range(1, 99999):
    try:
        
        webtoonlist = browser.find_element_by_xpath("//*[@id=\"contents\"]/div/div[2]/div/ul/li["+ str(num) +"]")
        # browser.implicitly_wait(3)
        webtoon = webtoonlist.find_element_by_xpath("div/a")
        #browser.execute_script("arguments[0].scrollIntoView(true);", webtoon) # Scroll to this webtoon element

        # 썸네일 주소
        thumb = webtoon.find_element_by_xpath('img')
        toon_thumb_url = thumb.get_attribute('data-original')
            # 성인 작품 여부 판단해서 pass
        if '_adult' in toon_thumb_url:
            num +=1
            continue
            
        
        # url
        toon_url = webtoon.get_attribute("href")
        # ID
        toon_id = 'mrblue_' + str(toon_url).replace("https://www.mrblue.com/webtoon/", '')
        
        textbox = webtoonlist.find_element_by_class_name("txt-box")
        
        # 제목
        title = textbox.find_element_by_xpath("span[1]/a")
        toon_name = title.get_attribute('title')
        # 장르
        genres = textbox.find_element_by_xpath("span[2]/span/a")
        genre = genres.get_attribute("title")
        
        # 작가
        writer = textbox.find_element_by_xpath("span[2]/a")
        wrt_name = writer.get_attribute('title')
            # 글/그림작가 다른 경우
        try:
            wrts = textbox.find_element_by_xpath("span[2]/a[2]")
            wrt_name += ', ' + wrts.get_attribute('title')
        except:
            pass

        '''
        # 설명
        # 얘도 마찬가지 이유로 뺌 ㅋ
        '''
        toon_desc = ''
        
        # toon_info
        cursor1.execute(sel_sql)
        result = cursor1.fetchall()
        if (toon_id not in result):
            cursor2.execute(ins_sql, (toon_id, toon_name, toon_url, toon_desc, 'mrblue', wrt_name, 'O', toon_thumb_url))
            cursor3.execute(genre_sql, (toon_id, genre))
        db.commit()

    except NoSuchElementException:
        break
        

In [6]:
# ------ 3. 회차 정보
#need to fill below webdriver path
#browser = webdriver.Chrome('C:/.../chromedriver.exe')

sel_url_sql = 'select toon_id, toon_url from toon_info where toon_site = "mrblue"'
sel_dup_sql = 'select toon_id, epi_name from epi_info where toon_id = %s and epi_name = %s'
ins_epi_info_sql = 'insert into epi_info(toon_id, epi_name, epi_url, epi_thumb_url, epi_date)\
                    values (%s, %s, %s, %s, %s)'

cursor1.execute(sel_url_sql)
result = cursor1.fetchall()
   
for row in result:
    toon_id = row['toon_id']
    toon_url = row['toon_url']
    
    browser.get(toon_url)
    #print('*********************************************')
    #print('<' + toon_id + '>')
    
    num = 1 # episode 개수 탐색
    while num < 99999:
        try:
            epilist = browser.find_element_by_class_name('view-list')
            #browser.execute_script("arguments[0].scrollIntoView(true);", episode) # Scroll to this episode
            #browser.implicitly_wait(3)
            
            episode = epilist.find_element_by_xpath("ul/li[" + str(num) + "]")
            page = episode.get_attribute('data-vol')
            
            # 썸네일 링크
            thumb = episode.find_element_by_xpath("div[2]/p/img")
            epi_thumb_url = thumb.get_attribute('data-original')
            
            # 에피소드 제목
            name = episode.find_element_by_xpath("div[3]/p/span")
            epi_name = name.text
            
            # URL
            epi_id = toon_id.replace("mrblue_", '')
            epi_url = "https://www.mrblue.com/viewer/view/" + epi_id + "/" + page + "?ppt=PPT02"
            
            # 업데이트 날짜 추출
            date = episode.find_element_by_xpath("div[3]/p[2]/span")
            epi_date = date.text.replace(".", "-")
            
            
            cursor2.execute(sel_dup_sql, (toon_id, epi_name))
            if cursor2.fetchall():
                num += 1
                continue
            else:
                cursor3.execute(ins_epi_info_sql, (toon_id, epi_name, epi_url, epi_thumb_url, epi_date))
                db.commit()
                
            num += 1
            
        except:
            break
db.close()